# Modelos Preditivos da Análise de Churn

## Contexto do Notebook

Neste notebook, abordarei a modelagem de aprendizado de máquina. Meu principal objetivo é desenvolver um modelo capaz de prever com precisão a probabilidade de um funcionário deixar a empresa. Com esse modelo, a empresa poderá planejar estratégias de retenção de funcionários, garantindo uma receita estável. Vale ressaltar que a aquisição de novos funcionários é mais custosa do que manter os atuais. Além disso, detalharei cada etapa abaixo, explicando minhas decisões em cada passo.

- ### Objetivo do Notebook

O objetivo principal deste Notebook é construir um modelo de machine learning capaz de prever com precisão a probabilidade de churn de funcionários. Isso permitirá que a empresa identifique os funcionários com maior probabilidade de deixar a empresa e implemente estratégias de retenção específicas para esses casos, reduzindo assim a taxa de churn e mantendo uma equipe estável e produtiva.

- ### Resultados Esperados

Espera-se que, ao final deste Notebook, tenhamos um modelo de machine learning capaz de prever com precisão a probabilidade de churn de funcionários. Isso permitirá que a empresa implemente estratégias proativas de retenção de funcionários, reduzindo assim a taxa de churn e mantendo uma equipe estável e produtiva.


## Importação das bibliotecas e base de dados

- Base de dados: https://www.kaggle.com/pavansubhasht/ibm-hr-analytics-attrition-dataset


In [21]:
# Visualização e Manipulação dos Dados
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Importação da base de dados
base = pd.read_csv('/content/Human_Resources.csv')

# Verificando quantos funcionários estão em Churn
count_values = base['Attrition'].value_counts()

# Calculando a porcentagem de Churn
churn_percentage = (count_values[1] / len(base)) * 100

# Mostrando os resultados de forma mais informativa
print(f"Total de funcionários na base de dados: {len(base)}")
print(f"Quantidade de funcionários em churn (valores 1): {count_values[1]}")
print(f"Quantidade de funcionários não em churn (valores 0): {count_values[0]}")
print(f"Porcentagem de Churn: {churn_percentage:.2f}%")




Total de funcionários na base de dados: 1470
Quantidade de funcionários em churn (valores 1): 237
Quantidade de funcionários não em churn (valores 0): 1233
Porcentagem de Churn: 16.12%


É evidente que há um desequilíbrio entre os funcionários, com cerca de 16% deixando a empresa e 84% permanecendo. Diante disso, é necessário buscar modelos e métricas de avaliação apropriados para lidar com essa análise e desequilíbrio.

## Realizando os Tratamentos Anteriores

In [22]:
# Transformando dados categoricos binarios
base['Attrition'] = base['Attrition'].apply(lambda x: 1 if x == 'Yes' else 0)
base['OverTime'] = base['OverTime'].apply(lambda x: 1 if x == 'Yes' else 0)

# Excluindo as colunas que não irão influenciar na análise
base.drop(['EmployeeCount', 'StandardHours', 'Over18', 'EmployeeNumber'], axis=1, inplace=True)

# Importando o OneHotEncoder
from sklearn.preprocessing import OneHotEncoder

# Criando o encoder
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore', dtype='int32')

# Lista das colunas a serem transformadas
columns_to_encode = ['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus']

# Iterando sobre as colunas e transformando-as
for col in columns_to_encode:
    ohe.fit(base[[col]])
    ohe_df = pd.DataFrame(ohe.transform(base[[col]]), columns=ohe.get_feature_names_out())
    base = pd.concat([base, ohe_df], axis=1)
    base.drop([col], axis=1, inplace=True)

# Verificando a base de dados resultante
base.head()

base.shape


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_outp

(1470, 51)

#### Divisão entre treino/teste

In [23]:
# Separando previsores e a coluna alvo

#Previsores
X = base[['BusinessTravel_Non-Travel', 'BusinessTravel_Travel_Frequently',
       'BusinessTravel_Travel_Rarely', 'Department_Human Resources',
       'Department_Research & Development', 'Department_Sales',
       'EducationField_Human Resources', 'EducationField_Life Sciences',
       'EducationField_Marketing', 'EducationField_Medical',
       'EducationField_Other', 'EducationField_Technical Degree',
       'Gender_Female', 'Gender_Male', 'JobRole_Healthcare Representative',
       'JobRole_Human Resources', 'JobRole_Laboratory Technician',
       'JobRole_Manager', 'JobRole_Manufacturing Director',
       'JobRole_Research Director', 'JobRole_Research Scientist',
       'JobRole_Sales Executive', 'JobRole_Sales Representative',
       'MaritalStatus_Divorced', 'MaritalStatus_Married',
       'MaritalStatus_Single','Age', 'DailyRate', 'DistanceFromHome',
       'Education', 'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement',
       'JobLevel', 'JobSatisfaction',	'MonthlyIncome',	'MonthlyRate',	'NumCompaniesWorked',	'OverTime',
       'PercentSalaryHike', 'PerformanceRating',	'RelationshipSatisfaction',	'StockOptionLevel',
       'TotalWorkingYears'	,'TrainingTimesLastYear'	, 'WorkLifeBalance',	'YearsAtCompany',
       'YearsInCurrentRole', 'YearsSinceLastPromotion',	'YearsWithCurrManager']]

#Coluna Alvo
y = base['Attrition']

#Normalizando os dados para que os algoritmos não considerem alguns atributos mais impotantes que outros:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Separando em treino/teste:
from sklearn.model_selection import train_test_split
X_treino, X_teste, y_treino, y_teste = train_test_split(X , y, test_size = 0.25, random_state=42)

Ao usar random_state=42, a divisão será sempre a mesma se usar o mesmo conjunto de dados X e y e manter o random_state como 42 em todas as chamadas de train_test_split. Se você alterar o valor de random_state, a divisão será diferente.

## Criação de Modelos

Modelos Escolhidos:
- Regressão Logística
- Random Forest

Esses modelos foram escolhidos por sua capacidade de lidar com classes desbalanceadas e por sua eficácia em problemas de classificação, como é o caso da preisão de Churn.

#### Importação das Bibliotecas

In [24]:
# Importação do modelo de regressão logística
from sklearn.linear_model import LogisticRegression

# Importação do modelo de classificação de Random Forest
from sklearn.ensemble import RandomForestClassifier

# Importação das métricas de avaliação de modelos
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
)

#### Regressão Logística

Optei por testar a regressão logística para modelar o churn dos funcionários pois:

1. Adequação à Variável de Resposta: A regressão logística é ideal para problemas em que a variável de resposta é binária, como é o caso do churn de funcionários, em que eles podem ou não deixar a empresa.

2. Interpretabilidade dos Resultados: A regressão logística oferece uma interpretação direta dos coeficientes como log-odds, o que significa que podemos entender facilmente o impacto de cada variável de entrada na probabilidade de churn de um funcionário.

3. Simplicidade e Eficiência: Este modelo é relativamente simples de implementar e computacionalmente eficiente para conjuntos de dados de tamanho moderado, como o nosso, facilitando a interpretação e aplicação prática dos resultados.

4. Tratamento do Desbalanceamento de Classes: A regressão logística pode lidar efetivamente com desbalanceamento de classes, uma vez que existem técnicas específicas, como a ponderação de classes ou o ajuste de limiares de decisão, que podem ser aplicadas para melhorar o desempenho do modelo nesse contexto.

Portanto, a escolha da regressão logística é fundamentada em sua capacidade de lidar com a natureza binária da variável de resposta, sua interpretabilidade direta, eficiência computacional e capacidade de tratar o **desbalanceamento** de classes, tornando-a uma escolha sólida para nosso problema de modelagem de churn de funcionários.

In [25]:
# Criando o modelo de regressão logística
reg_log = LogisticRegression(random_state=42)

# Treinando o modelo
reg_log.fit(X_treino, y_treino)

LogisticRegression(random_state=42)

In [26]:
# Fazendo a previsão
y_pred_rl = reg_log.predict(X_teste)
y_pred_rl

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

##### Métodos de Avaliação Regressão Logística:

Uma matriz de confusão é uma tabela que é usada para descrever o desempenho de um modelo de classificação em um conjunto de dados de teste para os quais os rótulos verdadeiros são conhecidos. A ideia é contar o número de vezes que cada classe real foi classificada como cada classe prevista pelo modelo. As entradas na matriz de confusão são divididas em quatro categorias:

[[TN, FP]

[FN, TP]]

Sendo:

- **Verdadeiro positivo (TP)**: Representa os casos em que o modelo previu corretamente que um cliente iria sair (churn) e de fato ele saiu. São importantes para entendermos a capacidade do modelo em identificar corretamente os casos positivos, ou seja, os clientes que realmente estão propensos a sair.

- **Falso positivo (FP)**: Indica os casos em que o modelo previu erroneamente que um cliente iria sair (churn), mas na verdade ele não saiu. Esses casos são relevantes, pois representam clientes que podem ser erroneamente identificados como propensos a sair, o que pode resultar em estratégias inadequadas de retenção de clientes.

- **Verdadeiro negativo (TN)**: São os casos em que o modelo previu corretamente que um cliente não iria sair (não churn) e de fato ele não saiu. Esses casos são importantes para entendermos a capacidade do modelo em identificar corretamente os casos negativos, ou seja, os clientes que não estão propensos a sair.

- **Falso negativo (FN)**: Representa os casos em que o modelo previu erroneamente que um cliente não iria sair (não churn), mas na verdade ele saiu. Esses casos são críticos, pois indicam clientes que foram erroneamente considerados como não propensos a sair, o que pode resultar em perda de oportunidades de retenção.

Na análise de Churn, essas classificações são usadas para calcular diversas métricas de avaliação do modelo, tais como precisão, recall, pontuação F1 e taxa de erro. Essas métricas ajudam a avaliar o desempenho do modelo em identificar corretamente os clientes propensos a sair (Churn) e os clientes não propensos a sair, fornecendo insights valiosos para ações de retenção de clientes.


In [27]:
# Calculando a matriz de confusão para avaliar o desempenho do modelo
cm = confusion_matrix(y_teste, y_pred_rl)
cm


array([[311,   9],
       [ 29,  19]])

Analisando a Matriz de Confusão:

- Verdadeiros Positivos (TP): 19
- Falsos Positivos (FP): 9
- Verdadeiros Negativos (TN): 311
- Falsos Negativos (FN): 29

Com esses valores, podemos calcular as seguintes métricas:

- Precisão: A precisão é a proporção de verdadeiros positivos em relação a todos os casos positivos previstos pelo modelo. É calculada como TP / (TP + FP). Neste caso, a precisão é 19 / (19 + 9) ≈ 0.68, ou 68%.

- Recall (Sensibilidade): O recall é a proporção de verdadeiros positivos em relação a todos os casos positivos reais. É calculado como TP / (TP + FN). Neste caso, o recall é 19 / (20 + 29) ≈ 0.4, ou aproximadamente 40%.

- F1-score(avg): O F1-score é a média harmônica da precisão e do recall e fornece um equilíbrio entre os dois. É calculado como 2 * (precisão * recall) / (precisão + recall). Neste caso, o F1-score é aproximadamente 0.72, ou 72%.

- Acurácia: A acurácia é a proporção de predições corretas em relação ao total de predições. É calculada como (TP + TN) / (TP + FP + TN + FN). Neste caso, a acurácia é (19 + 311) / (19 + 9 + 311+ 29) ≈ 0.90, ou aproximadamente 90%.

Essas métricas fornecem uma visão abrangente do desempenho do modelo de regressão logística na análise de churn. Em geral, parece que o modelo tem uma precisão bastante alta, mas um recall relativamente baixo, o que significa que ele pode estar identificando corretamente muitos dos casos de churn, mas também está perdendo alguns casos que deveriam ser identificados. Dependendo dos objetivos específicos da análise de churn, pode ser necessário ajustar o modelo para melhorar o recall ou outras métricas importantes.

Agora, irei gerar um relatório contendo essas métricas de avaliação e espera-se que os resultados estejam em conformidade com os cálculos realizados anteriormente.

In [28]:
# Relatório de Métricas
print(classification_report(y_teste, y_pred_rl))


              precision    recall  f1-score   support

           0       0.91      0.97      0.94       320
           1       0.68      0.40      0.50        48

    accuracy                           0.90       368
   macro avg       0.80      0.68      0.72       368
weighted avg       0.88      0.90      0.88       368



Como podemos notar, os resultados estão de acordo com os cálculos feitos anteriormente:

Relatório de análise de churn utilizando um modelo de Regressão Logística:

Os resultados da análise de churn utilizando o modelo Random Forest são apresentados a seguir:

**Para a classe 0 (funcionários que não deixaram a empresa):**

- A precisão é de 91%, indicando que 91% das previsões feitas para a classe 0 estão corretas.
- O recall é de 97%, o que significa que o modelo identificou corretamente 97% dos casos da classe 0.
- O F1-score é de 94%, uma métrica que representa a média harmônica entre precisão e recall, útil para avaliar o equilíbrio entre essas métricas.

**Para a classe 1 (funcionários que deixaram a empresa):**

- A precisão é de 80%, o que significa que 80% das previsões feitas para a classe 1 estão corretas.
- O recall é de 40%, indicando que o modelo identificou corretamente apenas 40% dos casos da classe 1.
- O F1-score é de 50%.
- A acurácia geral do modelo é de 90%, o que significa que ele classificou corretamente 90% dos exemplos.

A média ponderada das métricas (precisão, recall e F1-score) considerando o suporte de cada classe é de 88%, o que representa uma avaliação geral do modelo para ambas as classes, levando em conta o desbalanceamento entre elas.

Essas métricas fornecem uma visão abrangente do desempenho do modelo de Random Forest na análise de churn. Enquanto a precisão e o recall para a classe 0 são altos, para a classe 1, a precisão é razoável, mas o recall é baixo, indicando que o modelo pode estar tendo dificuldades em identificar corretamente os casos de churn. Sugere-se a exploração de outros modelos que possam fornecer um melhor desempenho de previsão.

Dessa forma, podemos notar que a acurácia, por si só, pode não ser uma boa métrica de avaliação para análise da saída de funcionários em casos onde apenas 16% dos funcionários deixam a empresa. Em um conjunto de dados desbalanceado como esse, um modelo pode atingir uma alta acurácia simplesmente prevendo a classe majoritária na maioria dos casos, ignorando completamente a classe minoritária. Isso ocorre porque a acurácia pode ser enganosa e não refletir a capacidade real do modelo de identificar os funcionários que saem da empresa.

Por outro lado, o recall e o F1-score são métricas mais adequadas para avaliar a efetividade do modelo de churn em um cenário desbalanceado. O recall, também conhecido como sensibilidade, mede a capacidade do modelo de identificar corretamente todos os casos positivos (Churn, neste caso) no conjunto de dados. Um recall baixo indicaria que o modelo está deixando de identificar muitos casos de churn, o que não é desejável em uma análise de churn.

O F1-score, que é a média harmônica entre precisão e recall, fornece um equilíbrio entre essas duas métricas. Ele é especialmente útil em conjuntos de dados desbalanceados, pois considera tanto a capacidade do modelo de identificar corretamente os casos positivos quanto a sua capacidade de evitar falsos positivos.

Portanto, em uma análise de Churn, é importante considerar métricas como recall e F1-score, que levam em conta a natureza desbalanceada do problema e fornecem uma avaliação mais precisa da efetividade do modelo.

Vou criar um DataFrame para armazenar as métricas de avaliação mais relevantes para uma análise de Churn: recall e F1-score. Posterormente irei atualizar esse DataFrame para comparar melhor outros tipos de modelo.

In [29]:
# Definir os dados do classification_report
data = {
    'recall_0': [0.97],
    'recall_1': [0.40],
    'f1_score_0': [0.94],
    'f1_score_1': [0.50]
}

# Criar o DataFrame
desempenho_modelos = pd.DataFrame(data, index=['regressao_logistica'])

# Exibir o DataFrame
print(desempenho_modelos)


                     recall_0  recall_1  f1_score_0  f1_score_1
regressao_logistica      0.97       0.4        0.94         0.5


#### Random Forest

Agora, com um entendimento mais robusto sobre as métricas de avaliação e interpretação dessas métricas, vou proceder com o teste do modelo Random Forest para modelar o Churn dos funcionários, motivado pelas seguintes razões principais:

1. Capacidade de Lidar com Dados Não Lineares: O Random Forest é capaz de capturar relações não lineares entre as variáveis de entrada e a variável de saída, o que é crucial em problemas complexos como a previsão de churn de funcionários, onde as interações entre os fatores podem ser não triviais.

2. Redução do Overfitting: O Random Forest tende a ter um desempenho melhor do que modelos lineares quando há uma grande quantidade de variáveis de entrada, pois ele seleciona aleatoriamente subconjuntos das variáveis em cada divisão da árvore, o que ajuda a reduzir o overfitting (quando um modelo se ajusta muito bem aos dados de treinamento, mas não consegue generalizar para novos dados, resultando em um desempenho inferior).

3. Robustez a Valores Ausentes e Outliers: O Random Forest é robusto a valores ausentes e outliers, o que é importante em conjuntos de dados do mundo real, onde esses problemas são comuns e podem prejudicar o desempenho de outros modelos.

4. Facilidade de Uso e Implementação: O Random Forest é relativamente simples de usar e não requer muitos ajustes de parâmetros para obter um bom desempenho, o que facilita sua implementação prática em nosso cenário.

5. Capacidade de Lidar com Desequilíbrio de Classes: O Random Forest pode lidar naturalmente com desequilíbrios de classes, como o que temos em nosso conjunto de dados, onde apenas cerca de 16% dos funcionários apresentam churn.

Portanto, a escolha do Random Forest é baseada em sua capacidade de lidar com relações não lineares, reduzir o overfitting, robustez a valores ausentes e outliers, facilidade de uso e implementação, e capacidade de lidar com **desequilíbrios de classes**, tornando-o uma escolha sólida e adequada para nosso problema de modelagem de churn de funcionários."

In [30]:
# Criando o modelo de classificação por florestas aleatórias
forest = RandomForestClassifier(random_state=42)

# Treinando o modelo
forest.fit(X_treino, y_treino)

RandomForestClassifier(random_state=42)

In [31]:
#Fazendo a previsão
y_pred_rf = forest.predict(X_teste)

In [32]:
# Calculando a matriz de confusão para avaliar o desempenho do modelo
cm_rf = confusion_matrix(y_pred_rf, y_teste)
cm_rf

array([[317,  45],
       [  3,   3]])

In [33]:
# Relatório de Métricas
print(classification_report(y_teste, y_pred_rf))

              precision    recall  f1-score   support

           0       0.88      0.99      0.93       320
           1       0.50      0.06      0.11        48

    accuracy                           0.87       368
   macro avg       0.69      0.53      0.52       368
weighted avg       0.83      0.87      0.82       368



##### Métodos de Avaliação Random Forest:

Relatório de análise de churn utilizando um modelo Random Forest:

Os resultados da análise de churn utilizando o modelo Random Forest são apresentados a seguir:

**Para a classe 0 ((funcionários que não deixaram a empresa)):**

- A precisão é de 88%, indicando que 88% das previsões feitas para a classe 0 estão corretas.
- O recall é de 99%, o que significa que o modelo identificou corretamente 99% dos casos da classe 0.
- O F1-score é de 93%, uma métrica que representa a média harmônica entre precisão e recall, útil para avaliar o equilíbrio entre essas métricas.

**Para a classe 1 ((funcionários que deixaram a empresa))**:

- A precisão é de 50%, o que significa que 50% das previsões feitas para a classe 1 estão corretas.
- O recall é de 6%, indicando que o modelo identificou corretamente apenas 6% dos casos da classe 1.
- O F1-score é de 11%.
- A acurácia geral do modelo é de 87%, o que significa que ele classificou corretamente 87% dos exemplos.

A média ponderada das métricas (precisão, recall e F1-score) considerando o suporte de cada classe é de 83%, o que representa uma avaliação geral do modelo para ambas as classes, levando em conta o desbalanceamento entre elas.

Essas métricas fornecem uma visão abrangente do desempenho do modelo de Random Forest na análise de churn. Em geral, a precisão e o recall para a classe 1(funcionários que deixaram a empresa) são baixos, o que indica que o modelo pode estar tendo dificuldades em identificar corretamente os casos de churn. Sugere-se a exploração de outros modelos que possam fornecer um melhor desempenho de previsão.

In [34]:
# Atualizando o DataFrame de desempenho
data_random_forest = {
    'recall_0': 0.99,
    'recall_1': 0.06,
    'f1_score_0': 0.93,
    'f1_score_1': 0.11
}


desempenho_modelos.loc['random_forest'] = data_random_forest

# Exibir o DataFrame atualizado
print(desempenho_modelos)

                     recall_0  recall_1  f1_score_0  f1_score_1
regressao_logistica      0.97      0.40        0.94        0.50
random_forest            0.99      0.06        0.93        0.11


### Outros Modelos

AdaBoost: Um algoritmo de boosting que pode ser eficaz para melhorar a performance em conjuntos de dados desbalanceados, dando mais peso às instâncias da classe minoritária.

In [35]:
from sklearn.ensemble import AdaBoostClassifier

# Criando o modelo de AdaBoost
adaboost = AdaBoostClassifier()

# Treinando o modelo
adaboost.fit(X_treino, y_treino)

# Fazendo previsões
y_pred = adaboost.predict(X_teste)


# Exibindo o relatório de classificação
print(classification_report(y_teste, y_pred))

# Matriz de Confusão
cm = confusion_matrix(y_pred, y_teste)
print(cm)

              precision    recall  f1-score   support

           0       0.90      0.94      0.92       320
           1       0.47      0.33      0.39        48

    accuracy                           0.86       368
   macro avg       0.69      0.64      0.66       368
weighted avg       0.85      0.86      0.85       368

[[302  32]
 [ 18  16]]


XGBoost: Esse algoritmo de gradient boosting que geralmente apresenta ótima performance em conjuntos de dados desbalanceados, mas pode exigir um pouco mais de ajuste de hiperparâmetros.

In [36]:
from xgboost import XGBClassifier

# Criando o modelo XGBoost
xgb = XGBClassifier()

# Treinando o modelo
xgb.fit(X_treino, y_treino)

# Fazendo previsões
y_pred = xgb.predict(X_teste)

# Exibindo o relatório de classificação
print(classification_report(y_teste, y_pred))

# Matriz de Confusão
cm = confusion_matrix(y_pred, y_teste)
print(cm)

              precision    recall  f1-score   support

           0       0.90      0.97      0.94       320
           1       0.62      0.31      0.42        48

    accuracy                           0.89       368
   macro avg       0.76      0.64      0.68       368
weighted avg       0.87      0.89      0.87       368

[[311  33]
 [  9  15]]


SVM (Support Vector Machines): O SVM pode ser eficaz para problemas de classificação binária como churn, especialmente com kernels apropriados.

In [37]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Criando o modelo SVM
svm = SVC()

# Treinando o modelo
svm.fit(X_treino, y_treino)

# Fazendo previsões
y_pred = svm.predict(X_teste)

# Exibindo o relatório de classificação
print(classification_report(y_teste, y_pred))

# Matriz de Confusão
cm = confusion_matrix(y_pred, y_teste)
print(cm)

              precision    recall  f1-score   support

           0       0.90      0.99      0.95       320
           1       0.87      0.27      0.41        48

    accuracy                           0.90       368
   macro avg       0.88      0.63      0.68       368
weighted avg       0.90      0.90      0.88       368

[[318  35]
 [  2  13]]


KNN (K-Nearest Neighbors): O KNN é um modelo simples que pode ser eficaz para detectar padrões em conjuntos de dados desbalanceados.

In [38]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

# Criando o modelo KNN
knn = KNeighborsClassifier()

# Treinando o modelo
knn.fit(X_treino, y_treino)

# Fazendo previsões
y_pred = knn.predict(X_teste)

# Exibindo o relatório de classificação
print(classification_report(y_teste, y_pred))

# Matriz de Confusão
cm = confusion_matrix(y_pred, y_teste)
print(cm)

              precision    recall  f1-score   support

           0       0.89      0.99      0.94       320
           1       0.77      0.21      0.33        48

    accuracy                           0.89       368
   macro avg       0.83      0.60      0.63       368
weighted avg       0.88      0.89      0.86       368

[[317  38]
 [  3  10]]


Naive Bayes: O Naive Bayes é um modelo probabilístico simples que pode funcionar bem em conjuntos de dados desbalanceados. Embora não seja tão complexo, pode ser eficaz para aumentar o recall.

In [39]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

# Criando o modelo Naive Bayes
nb = GaussianNB()

# Treinando o modelo
nb.fit(X_treino, y_treino)

# Fazendo previsões
y_pred = nb.predict(X_teste)

# Exibindo o relatório de classificação
print(classification_report(y_teste, y_pred))

# Matriz de Confusão
cm = confusion_matrix(y_pred, y_teste)
print(cm)

              precision    recall  f1-score   support

           0       0.92      0.68      0.78       320
           1       0.22      0.60      0.32        48

    accuracy                           0.67       368
   macro avg       0.57      0.64      0.55       368
weighted avg       0.83      0.67      0.72       368

[[218  19]
 [102  29]]


Atualizarei a tabela de desempenho dos modelos preditivos

In [40]:
# Atualizando o DataFrame de desempenho

data_adaboost = {
    'recall_0': 0.94,
    'recall_1': 0.33,
    'f1_score_0': 0.92,
    'f1_score_1': 0.39
}


desempenho_modelos.loc['adaboost'] = data_adaboost

data_xgb = {
    'recall_0': 0.97,
    'recall_1': 0.31,
    'f1_score_0': 0.94,
    'f1_score_1': 0.42
}


desempenho_modelos.loc['xgb'] = data_xgb


data_svm = {
    'recall_0': 0.99,
    'recall_1': 0.27,
    'f1_score_0': 0.95,
    'f1_score_1': 0.41
}


desempenho_modelos.loc['svm'] = data_svm

data_knn = {
    'recall_0': 0.99,
    'recall_1': 0.21,
    'f1_score_0': 0.94,
    'f1_score_1': 0.33
}


desempenho_modelos.loc['knn'] = data_knn

data_nb = {
    'recall_0': 0.68,
    'recall_1': 0.60,
    'f1_score_0': 0.78,
    'f1_score_1': 0.32
}


desempenho_modelos.loc['nb'] = data_nb



# Exibir o DataFrame atualizado
print(desempenho_modelos)

                     recall_0  recall_1  f1_score_0  f1_score_1
regressao_logistica      0.97      0.40        0.94        0.50
random_forest            0.99      0.06        0.93        0.11
adaboost                 0.94      0.33        0.92        0.39
xgb                      0.97      0.31        0.94        0.42
svm                      0.99      0.27        0.95        0.41
knn                      0.99      0.21        0.94        0.33
nb                       0.68      0.60        0.78        0.32


### Relatório dos Modelos

**Regressão Logística:**

- Recall Classe 0: 0.97
- Recall Classe 1 (Churn): 0.40
- F1-Score Classe 0: 0.94
- F1-Score Classe 1 (Churn): 0.50

**Random Forest:**

- Recall Classe 0: 0.99
- Recall Classe 1 (Churn): 0.06
- F1-Score Classe 0: 0.93
- F1-Score Classe 1 (Churn): 0.11

**AdaBoost:**

- Recall Classe 0: 0.94
- Recall Classe 1 (Churn): 0.33
- F1-Score Classe 0: 0.92
- F1-Score Classe 1 (Churn): 0.39

**XGBoost:**

- Recall Classe 0: 0.97
- Recall Classe 1 (Churn): 0.31
- F1-Score Classe 0: 0.94
- F1-Score Classe 1 (Churn): 0.42

**SVM:**

- Recall Classe 0: 0.99
- Recall Classe 1 (Churn): 0.27
- F1-Score Classe 0: 0.95
- F1-Score Classe 1 (Churn): 0.41

**KNN:**

- Recall Classe 0: 0.99
- Recall Classe 1 (Churn): 0.21
- F1-Score Classe 0: 0.94
- F1-Score Classe 1 (Churn): 0.33

**Naive Bayes:**

- Recall Classe 0: 0.68
- Recall Classe 1 (Churn): 0.60
- F1-Score Classe 0: 0.78
- F1-Score Classe 1 (Churn): 0.32

**Conclusão e Próximos Passos:**

Com base na análise dos resultados, os algoritmos que se destacam são a Regressão Logística e o XGBoost. Ambos apresentaram um bom equilíbrio entre recall e f1-score para a classe de interesse (churn).

Considerando a simplicidade e interpretabilidade da Regressão Logística, ela será uma boa opção para tentar otimizar inicialmente. Já o XGBoost, apesar de ser mais complexo, também mostrou um bom desempenho e capacidade de lidar com conjuntos de dados desbalanceados, sendo outra opção promissora para otimização.

O próximo passo será realizar a otimizção dos modelos. Usarei técnicas como balanceamento de classes e seleção de atributos para melhorar ainda mais seu desempenho na identificação de casos de churn. Essa otimização está contida nesse [notebook]()
